In [13]:
import pandas as pd
import sqlite3
import xgboost as xgb
from pathlib import Path
from tqdm.notebook import tqdm
from dateutil.relativedelta import relativedelta


In [14]:

def dbPath() -> Path:
    cwd = Path.cwd()
    dc2 = cwd.parent
    db = dc2.joinpath("data/database_final.db")
    return db


def modelPath(lsoaCode:str) ->Path:
    cwd = Path.cwd()
    dc2 = cwd.parent
    modelPath = dc2.joinpath(f"data/models/{lsoaCode}.bin")
    return modelPath


In [15]:
conn = sqlite3.connect(dbPath())
query_lsoa_codes = "SELECT geogcode FROM lsoa_code_to_name"
lsoa_codes = pd.read_sql(query_lsoa_codes, conn)

query_unemployement = "SELECT * FROM montly_unemployement_claimant_count_by_lsoa_barnet"
unemployement = pd.read_sql(query_unemployement, conn)

unemployement.dropna(inplace=True)
unemployement.drop(columns="index", inplace=True)
unemployement = unemployement[(unemployement["date"]<"2020") & (unemployement["date"]>"2012")]

unemployement = unemployement[unemployement["geogcode"]!="Column Total"].copy()
lsoa_code_list = lsoa_codes[lsoa_codes["geogcode"] != "Column Total"]["geogcode"].to_list()


def makeAllFeatures(unemployement,lsoa_code_list):
    allFeatures = pd.DataFrame(columns=lsoa_code_list, index=pd.to_datetime(unemployement["date"].unique()))

    for row in tqdm(unemployement.index):
        allFeatures[unemployement["geogcode"][row]][unemployement["date"][row]] = unemployement["value"][row]

    shiftColumnList = []
    for code in tqdm(lsoa_code_list):
        tempDF = pd.DataFrame(columns=[f"{code}_shift_{i+1}" for i in range(12)])
        for i in range(12):
            # allFeatures[f"{code}_shift_{i+1}"] = allFeatures[f"{code}"].shift(i+1)
            tempDF[f"{code}_shift_{i+1}"] = allFeatures[code].shift(i+1)
        shiftColumnList.append(tempDF.copy())
    allShifts = pd.concat(shiftColumnList, axis=1)
    # return allShifts, 0
    allFeatures = pd.concat([allFeatures, allShifts], axis=1)
    allFeatures = allFeatures.dropna()
    conn.close()
    return allFeatures

allFeatures = makeAllFeatures(unemployement, lsoa_code_list)

  0%|          | 0/17724 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

In [16]:
allFeatures

,E01000115,E01000116,E01000117,E01000118,E01000119,E01000120,E01000121,E01000122,E01000123,E01000124,...,E01033573_shift_3,E01033573_shift_4,E01033573_shift_5,E01033573_shift_6,E01033573_shift_7,E01033573_shift_8,E01033573_shift_9,E01033573_shift_10,E01033573_shift_11,E01033573_shift_12
2014-01-01,10.0,15.0,20.0,20.0,15.0,15.0,35.0,15.0,20.0,45.0,...,35.0,35.0,35.0,40.0,35.0,30.0,30.0,30.0,30.0,25.0
2014-02-01,10.0,15.0,20.0,25.0,20.0,20.0,35.0,15.0,20.0,45.0,...,30.0,35.0,35.0,35.0,40.0,35.0,30.0,30.0,30.0,30.0
2014-03-01,10.0,10.0,15.0,20.0,15.0,15.0,35.0,15.0,25.0,40.0,...,30.0,30.0,35.0,35.0,35.0,40.0,35.0,30.0,30.0,30.0
2014-04-01,10.0,15.0,10.0,20.0,15.0,20.0,25.0,20.0,15.0,40.0,...,20.0,30.0,30.0,35.0,35.0,35.0,40.0,35.0,30.0,30.0
2014-05-01,10.0,15.0,15.0,20.0,10.0,20.0,30.0,15.0,15.0,45.0,...,25.0,20.0,30.0,30.0,35.0,35.0,35.0,40.0,35.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,10.0,10.0,15.0,20.0,10.0,10.0,30.0,10.0,20.0,45.0,...,40.0,30.0,30.0,30.0,35.0,35.0,25.0,30.0,30.0,25.0
2019-09-01,10.0,5.0,15.0,15.0,10.0,10.0,25.0,10.0,15.0,45.0,...,40.0,40.0,30.0,30.0,30.0,35.0,35.0,25.0,30.0,30.0
2019-10-01,10.0,5.0,15.0,20.0,15.0,15.0,30.0,10.0,15.0,45.0,...,40.0,40.0,40.0,30.0,30.0,30.0,35.0,35.0,25.0,30.0
2019-11-01,10.0,5.0,15.0,20.0,15.0,10.0,30.0,15.0,20.0,50.0,...,40.0,40.0,40.0,40.0,30.0,30.0,30.0,35.0,35.0,25.0


In [17]:
unemployement

,geogcode,date,value
324,E01000248,2013-01-01 00:00:00,15.0
325,E01000248,2013-02-01 00:00:00,10.0
326,E01000248,2013-03-01 00:00:00,10.0
327,E01000248,2013-04-01 00:00:00,5.0
328,E01000248,2013-05-01 00:00:00,5.0
...,...,...,...
94483,E01000143,2019-08-01 00:00:00,40.0
94484,E01000143,2019-09-01 00:00:00,40.0
94485,E01000143,2019-10-01 00:00:00,40.0
94486,E01000143,2019-11-01 00:00:00,40.0


In [18]:
for i in range(3):
    row = allFeatures[allFeatures.index == list(allFeatures.index)[-1]]
    temp = pd.to_datetime(list(row.index)[0])
    print(temp)
    pd.to_datetime(temp+relativedelta(months=1))
    guessList = []
    for code in lsoa_code_list:
        model = xgb.XGBRegressor()
        model.load_model(modelPath(code))
        arr = row.values
        arr = arr.reshape(-1,1)
        arr = arr[211:]
        arr = arr.reshape(1,-1)
        guess = model.predict(arr)[0]
        dct = {"geogcode":code, "date": pd.to_datetime(temp+relativedelta(months=1)), "value":guess}
        guessList.append(dct)

    new = pd.DataFrame(guessList)


    unemployement = pd.concat([unemployement, new])
    allFeatures= makeAllFeatures(unemployement, lsoa_code_list)

2019-12-01 00:00:00


  0%|          | 0/17935 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

2020-01-01 00:00:00


  0%|          | 0/18146 [00:00<?, ?it/s]

NotImplementedError: 

In [ ]:
allFeatures.tail()

In [78]:
print(len(unemployement["date"].unique()))
for i in tqdm(range(12)):
    allFeatures = makeAllFeatures(unemployement.copy(), lsoa_code_list)
    final_row = allFeatures[allFeatures.index == list(allFeatures.index)[-1]]
    date = pd.to_datetime(list(row.index)[0])
    guessDate = pd.to_datetime(date+relativedelta(months=1))


    guessList = []
    for code in lsoa_code_list:
        model = xgb.XGBRegressor()
        model.load_model(modelPath(code))
        arr = row.values
        arr = arr.reshape(-1,1)
        arr = arr[211:]
        arr = arr.reshape(1,-1)
        guess = model.predict(arr)[0]
        dct = {"geogcode":code, "date": pd.to_datetime(temp+relativedelta(months=1)), "value":guess}
        guessList.append(dct)
    new = pd.DataFrame(guessList)
    unemployement = pd.concat([unemployement, new])

print(len(unemployement["date"].unique()))



84


  0%|          | 0/17724 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/17935 [00:00<?, ?it/s]

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/18146 [00:00<?, ?it/s]

NotImplementedError: 